In [1]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pid
from polars_indicators import IndicatorResult
DATE_COLUNN = "Date"

In [2]:
symbols = ['A','AA']

In [3]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2021, 1, 1))
df = df.collect()

In [4]:
df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol
date,f64,f64,f64,f64,f64,i64,str
2021-01-04,118.940002,120.089996,117.110001,118.639999,117.230392,2030700,"""A"""
2021-01-05,118.480003,120.32,118.07,119.610001,118.188873,2344000,"""A"""
2021-01-06,118.690002,123.879997,118.209999,122.889999,121.429886,2428500,"""A"""
2021-01-07,124.330002,127.480003,124.110001,126.160004,124.661041,1775900,"""A"""
2021-01-08,126.489998,127.639999,125.849998,127.059998,125.550339,1577200,"""A"""
2021-01-11,126.449997,129.339996,126.230003,128.149994,126.627388,1746800,"""A"""
2021-01-12,128.330002,128.389999,126.199997,126.970001,125.461426,1826400,"""A"""
2021-01-13,127.459999,128.050003,126.739998,126.980003,125.471298,1530400,"""A"""
2021-01-14,127.269997,128.190002,125.82,126.029999,124.532593,924900,"""A"""


In [5]:
sma25 = pi.simple_moving_average(df, days=25)
sma50 = pi.simple_moving_average(sma25.df, days=50)
df = sma50.df
df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA25,SMA50
date,f64,f64,f64,f64,f64,i64,str,f64,f64
2021-01-04,118.940002,120.089996,117.110001,118.639999,117.230392,2030700,"""A""",null,null
2021-01-05,118.480003,120.32,118.07,119.610001,118.188873,2344000,"""A""",null,null
2021-01-06,118.690002,123.879997,118.209999,122.889999,121.429886,2428500,"""A""",null,null
2021-01-07,124.330002,127.480003,124.110001,126.160004,124.661041,1775900,"""A""",null,null
2021-01-08,126.489998,127.639999,125.849998,127.059998,125.550339,1577200,"""A""",null,null
2021-01-11,126.449997,129.339996,126.230003,128.149994,126.627388,1746800,"""A""",null,null
2021-01-12,128.330002,128.389999,126.199997,126.970001,125.461426,1826400,"""A""",null,null
2021-01-13,127.459999,128.050003,126.739998,126.980003,125.471298,1530400,"""A""",null,null
2021-01-14,127.269997,128.190002,125.82,126.029999,124.532593,924900,"""A""",null,null


In [6]:
cross_up = pi.crossover_up(df, sma25.column, sma50.column)
cross_up.df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA25,SMA50,SMA25_cross_up_SMA50
date,f64,f64,f64,f64,f64,i64,str,f64,f64,bool
2021-01-04,118.940002,120.089996,117.110001,118.639999,117.230392,2030700,"""A""",null,null,false
2021-01-05,118.480003,120.32,118.07,119.610001,118.188873,2344000,"""A""",null,null,null
2021-01-06,118.690002,123.879997,118.209999,122.889999,121.429886,2428500,"""A""",null,null,null
2021-01-07,124.330002,127.480003,124.110001,126.160004,124.661041,1775900,"""A""",null,null,null
2021-01-08,126.489998,127.639999,125.849998,127.059998,125.550339,1577200,"""A""",null,null,null
2021-01-11,126.449997,129.339996,126.230003,128.149994,126.627388,1746800,"""A""",null,null,null
2021-01-12,128.330002,128.389999,126.199997,126.970001,125.461426,1826400,"""A""",null,null,null
2021-01-13,127.459999,128.050003,126.739998,126.980003,125.471298,1530400,"""A""",null,null,null
2021-01-14,127.269997,128.190002,125.82,126.029999,124.532593,924900,"""A""",null,null,null


In [7]:
#convert to open values
cross_up.df = cross_up.df.with_columns(pl.when(pl.col(cross_up.column)).then(pl.col("Open")).alias(cross_up.column))

In [8]:
trail = pi.trailing_stop(cross_up.df, bars=2)
t = trail.df.filter(pl.col("Date").is_between(datetime(2022, 6, 1), datetime(2022, 6, 18)) & (pl.col("Symbol") == "A"))
t

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA25,SMA50,SMA25_cross_up_SMA50,2_bar_trailing_stop
date,f64,f64,f64,f64,f64,i64,str,f64,f64,f64,f64
2022-06-01,127.860001,128.979996,122.43,123.089996,122.477409,1757600,"""A""",121.4512,126.123399,null,124.489998
2022-06-02,122.959999,129.190002,122.300003,129.100006,128.457504,1444400,"""A""",121.908,125.9196,null,122.43
2022-06-03,127.620003,128.720001,126.419998,128.490005,127.850555,1722800,"""A""",122.205201,125.8058,null,null
2022-06-06,129.380005,130.0,126.970001,127.629997,126.99482,1310600,"""A""",122.539601,125.6302,null,null
2022-06-07,126.57,131.199997,126.57,128.899994,128.258499,1236300,"""A""",122.9128,125.493999,null,null
2022-06-08,127.879997,128.639999,126.400002,126.839996,126.208755,1049500,"""A""",123.0904,125.322399,null,126.57
2022-06-09,126.089996,126.919998,124.129997,124.139999,123.522186,814400,"""A""",123.004,125.036799,null,126.089996
2022-06-10,121.629997,121.93,119.900002,120.57,119.969955,1379600,"""A""",122.9408,124.738999,null,121.629997
2022-06-13,116.480003,117.699997,115.379997,116.279999,115.701309,1405800,"""A""",122.7488,124.417999,null,116.480003


In [10]:
index = "row_nr"
column = "vals"
bars = 2
values = [0.1, 0.2, None, None, None, 1.1, 1.2, 1.3, 1.4, 1.5, None, 1.7]
ind = range(len(values))
d = {index: ind,
     column: values}
df = pl.DataFrame(d)
df

row_nr,vals
i64,f64
0,0.1
1,0.2
2,null
3,null
4,null
5,1.1
6,1.2
7,1.3
8,1.4


In [11]:
tes = "tes"
df = df.with_columns(pl.lit(True).alias(tes))
df

row_nr,vals,tes
i64,f64,bool
0,0.1,true
1,0.2,true
2,null,true
3,null,true
4,null,true
5,1.1,true
6,1.2,true
7,1.3,true
8,1.4,true


In [12]:

df = df.with_columns(pl.col(tes) & pl.col(column).shift(2).is_null().alias(tes))
df

row_nr,vals,tes
i64,f64,bool
0,0.1,true
1,0.2,true
2,null,false
3,null,false
4,null,true
5,1.1,true
6,1.2,true
7,1.3,false
8,1.4,false


In [13]:
df = df.with_columns(pl.col(tes) & pl.col(column).shift(1).is_null().alias(tes))
df

row_nr,vals,tes
i64,f64,bool
0,0.1,true
1,0.2,false
2,null,false
3,null,false
4,null,true
5,1.1,true
6,1.2,false
7,1.3,false
8,1.4,false


In [14]:
df.with_columns(pl.when(pl.col(tes)).then(1).otherwise(0).cumsum().alias(col1)) 

NameError: name 'col1' is not defined

In [15]:
col1 = "col1"
df = df.with_columns(pl.when(pl.col(column).shift(2).is_null() & pl.col(column).shift(1).is_null()).then(1).otherwise(0).cumsum().alias(col1)) 
df

row_nr,vals,tes,col1
i64,f64,bool,i32
0,0.1,true,1
1,0.2,false,1
2,null,false,1
3,null,false,1
4,null,true,2
5,1.1,true,3
6,1.2,false,3
7,1.3,false,3
8,1.4,false,3


In [16]:
col2 = "col2"
ones = "ones"
df = df.with_columns(pl.lit(1).alias(ones)).with_columns(pl.col(ones).cumsum().over(col1).alias(col2))
df

row_nr,vals,tes,col1,ones,col2
i64,f64,bool,i32,i32,i32
0,0.1,true,1,1,1
1,0.2,false,1,1,2
2,null,false,1,1,3
3,null,false,1,1,4
4,null,true,2,1,1
5,1.1,true,3,1,1
6,1.2,false,3,1,2
7,1.3,false,3,1,3
8,1.4,false,3,1,4


In [17]:
df.with_columns(pl.when(pl.col(col2) % 3 != 1).then(0).otherwise(pl.col(col2)))

row_nr,vals,tes,col1,ones,col2,literal
i64,f64,bool,i32,i32,i32,i32
0,0.1,true,1,1,1,1
1,0.2,false,1,1,2,0
2,null,false,1,1,3,0
3,null,false,1,1,4,4
4,null,true,2,1,1,1
5,1.1,true,3,1,1,1
6,1.2,false,3,1,2,0
7,1.3,false,3,1,3,0
8,1.4,false,3,1,4,4


In [13]:
def group_by_bars(df: pl.DataFrame | pl.LazyFrame, nulls: int, column: str) -> IndicatorResult:
    """groups trades together allowing for a specified number of nulls in each group"""
    column_name = f"group_{nulls}"
    if column_name in df.columns:
        return IndicatorResult(df, column_name)
    
    tes = "tes"
    df = df.with_columns(pl.lit(True).alias(tes))
    
    for i in range(1, nulls+1):
        df = df.with_columns(pl.col(tes) & pl.col(column).shift(i).is_null().alias(tes))

    df = df.with_columns(pl.when(pl.col(tes)).then(1).otherwise(0).cumsum().alias(column_name)) 

    return IndicatorResult(df, column_name)
def limit_entries(df: pl.DataFrame | pl.LazyFrame, bars: int, entries: str) -> IndicatorResult:
    """Forces a minimum number of bars between entries"""
    column_name = f"{bars}_minimum_bars_between"
    if column_name in df.columns:
        return IndicatorResult(df, column_name)
    
    group = group_by_bars(df, bars, entries)

    ones = "ones"

    df = group.df

    sum = "sum"
    df = group.df.with_columns(pl.lit(1).alias(ones)).with_columns(pl.col(ones).cumsum().over(group.column).alias(sum))

    df = df.with_columns(pl.when(pl.col(sum) % (bars+1) == 1).then(pl.col(entries)).alias(column_name))


    return IndicatorResult(df, column_name)

index = "row_nr"
column = "vals"
values = [0.1, 0.2, None, None, None, 1.1, 1.2, 1.3, 1.4, 1.5, None, 1.7]
ind = range(len(values))
d = {index: ind,
     column: values}
df = pl.DataFrame(d)
df
group_by_bars(df, 2, column).df
limit_entries(df, 2, column).df

row_nr,vals,tes,group_2,ones,sum,2_minimum_bars_between
i64,f64,bool,i32,i32,i32,f64
0,0.1,true,1,1,1,0.1
1,0.2,false,1,1,2,null
2,null,false,1,1,3,null
3,null,false,1,1,4,null
4,null,true,2,1,1,null
5,1.1,true,3,1,1,1.1
6,1.2,false,3,1,2,null
7,1.3,false,3,1,3,null
8,1.4,false,3,1,4,1.4


In [ ]:
ret = pi.create_trade_ids(trail.df, cross_up.column, trail.column)

In [ ]:
pi.summarize_trades(ret.df, ret.column, cross_up.column, trail.column)